In [71]:
# Required libraries to run code 
import pandas as pd
import numpy as np
import regex as re
import time
import os

from tqdm.auto import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import hnswlib
from scipy.spatial import distance
from datasketch import MinHash, MinHashLSHForest,MinHashLSHEnsemble

import warnings
warnings.filterwarnings("ignore")

## Load the main data

In [72]:
df = pd.read_csv('Data/wi_dataset.csv', lineterminator='\n', parse_dates=['retrieval_date'])

# SentenceRemover 
- SentenceRemover is a collection of methods developed to remove part of offer that occurs many times in others e.g. (text not connected with real job description)
- It is based on IDF to measure how many times a 5-gram occurs within job offers for each language.
- As a preprocessing step we used some regular expressions and make use of LCS (longest common substring) to detect part of offer that was duplicated inside description to ensure data quality for creation of embeddings

## Initialize shared methods for Sentence Remover

In [4]:
def find_dup_longest_common_substring(text):
    """Find duplicated the longest common substring(sequence of words) inside text"""
    """To speed up computation instead of comparing chars we use entire tokens(words separated by space)"""
    
    # Split the text into words
    words = text.split(" ")
    n = len(words)
    
    # Initialize a matrix to store the lengths of longest common suffixes
    matrix = [[0] * (n + 1) for _ in range(n + 1)]
    
    # Initialize variables to store the length and position of the longest common substring
    max_length = 0
    end_pos = 0
    
    # Iterate over all possible pairs of suffixes
    for i in range(1, n + 1):
        for j in range(i + 1, n + 1):
            
            # Check if the suffixes match
            if words[i - 1] == words[j - 1]:
                # If they do, update the matrix entry and check if it's the longest match so far
                matrix[i][j] = matrix[i - 1][j - 1] + 1
                if matrix[i][j] > max_length:
                    max_length = matrix[i][j]
                    end_pos = i
            
            # If the suffixes don't match, set the matrix entry to 0
            else:
                matrix[i][j] = 0
    
    # Use the end position and length of the longest match to return the matching substring
    return " ".join(words[end_pos - max_length:end_pos])


def custom_tokenizer(text):
    """The function that will be used(in TfidfVectorizer) to convert the input text into a sequence of tokens 
    (i.e., words or subwords) that will be used as features in the resulting document-term matrix."""
    # Remove digits from the text
    text = re.sub(r'[\d]+', '', text)
    # Remove punctuation from the text
    text = re.sub(r"""[!"#$%&\\'()*+,\-./:;<=>?@[\]^_`{|}~]+""", ' ', text)
    # Remove sequences of the same character (at least 3 in a row)
    text = re.sub(r"(\w)\1{2,}" , "", text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Find all word tokens in the text
    tokens = re.findall(r"(?u)\b\w\w+\b", text)
    
    return tokens


def remove_duplicate_sentences(text):
    """Function that takes in the raw input text and applies preprocessing steps to it before tokenizing (in TfidfVectorizer)"""
    text = re.sub(r'\\n', '', text)  # Remove newline characters
    text = re.sub(r"""([\w!"#$%&\\'()*+,\-./:;<=>?@[\]^_`{|}~])\1{2,}""" , "", text)  # Remove many of the same character (at least 3 in a row)
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    
    sub = find_dup_longest_common_substring(text)  # Find the longest duplicated substring
    if len(sub.split()) > 15:  # If the substring is longer than 15 words
        text = text.replace(sub, " ", 1)  # Replace only the first occurrence of the substring with a space
    return text  # Return the modified text

def buildNgramDictionary(vectorizer,country):
    """
    Function designed to create dictionary which stores the most popular 5-grams to be removed.
    
    Args:
    - vectorizer: a vectorizer object with IDF weights already computed
    - country: a string representing the country for which the dictionary is being built
    
    Returns:
    - terms_dict: a dictionary containing the n-grams with IDF values smaller than the chosen quantile
    """
    
    # Get the feature names and idf values from the vectorizer
    feature_names = vectorizer.get_feature_names_out()
    idf_values = vectorizer.idf_
    
    # Remove duplicate idf values and sort the array in descending order
    unique_idf = list(set(idf_values))
    sorted_idf = np.sort(unique_idf)[::-1]
    
    # Compute the three largest values in the array
    largest_values = sorted_idf[:3]
    
    # Set the initial quantile value to 0.02
    q = 0.02
    idf_quantile = 0
    
    # If the job offers are from Luxembourg, set the threshold manually
    if country == 'LU':
        q = 0.0001
    else:
        # Compute the quantile value
        q_value = np.quantile(idf_values, q)
        
        # Check if the quantile value is different from the three largest values
        if q_value not in largest_values:
            idf_quantile = np.quantile(idf_values, q)
        else:
            #set third highest value
            idf_quantile = largest_values[-1]

    # Create a dictionary with only the terms that have an idf value smaller than the quantile
    terms_dict = {}
    for i, term in enumerate(feature_names):
        if idf_values[i] <= idf_quantile:
            terms_dict[term] = idf_values[i]
            
    return terms_dict


def removeTrashSentencesIDFv2(document, vectorizer_dict):
    """It takes in two arguments:
    1. document - a string representing the text to be cleaned.
    2. vectorizer_dict - a dictionary containing for each language the 'TfidfVectorizer' object; 
    built tfidf_vectorizer.build_preprocessor() and tfidf_vectorizer.build_tokenizer() function; the dictionary with 5-gram to remove.
    """
    
    # If the document is a string, proceed with the cleaning process.
    if(not isinstance(document,str)):
        return None

    else:
        if document in short_desc_cache:#is saved copy
            return short_desc_cache[document]
        else:
            # Get the dictionary of 5-grams to be removed.
            terms_dict = vectorizer_dict['terms_dict']
            removal_ = terms_dict.keys()#5grams to remove
            
            # Apply the preprocessor function to the document.
            document_pre=vectorizer_dict['vectorizerPreprocessor'](document)
            
          
            # Remove identified 5-grams from text using a generator expression and a regular expression.
            # The pattern is defined as - (...|ngram|...) where ngram is each 5-gram to be removed and the 
            # text is every token joined with space.
            cleaned_text = re.sub(r'\b{}\b'.format('|'.join(ngram for ngram in removal_)), '', ' '.join(word for word in vectorizer_dict['vectorizerTokenizer'](document_pre)))
            
            # Replace all consecutive whitespaces with a single whitespace.
            cleaned_text  =re.sub(r"\s+"," ",cleaned_text)
            
            # If the cleaned_text is empty, return the original document.
            if cleaned_text=="":
                cleaned_text = document
            
            #save short_desc inside dictionary for reuse
            short_desc_cache[document] = cleaned_text
            # Otherwise, return the cleaned_text.
            return cleaned_text


### Make separate vocabulary (of 5-grams to remove) based on IDF for each language

In [6]:
vectorizer_dict={}#stores vectorizer for each language

In [7]:
countryList = df.country_id.unique()

In [8]:
#Iterate through each country in the list of countries
for country in tqdm(countryList):
    # Create a TfidfVectorizer object with 5-grams, remove duplicate sentences, and custom tokenizer
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(5, 5), preprocessor=remove_duplicate_sentences, dtype='float32', tokenizer=custom_tokenizer)
    
    # Filter the dataframe by the current country, drop duplicate descriptions, drop rows with missing descriptions,
    # and reset the index
    tfIDF = df.query(f"country_id == '{country}'").drop_duplicates(['description']).dropna(subset=['description']).reset_index(drop=True)
    
    # Fit the TfidfVectorizer object to the descriptions of the filtered dataframe
    tfidf_vectorizer.fit(tfIDF['description'])
    
    # Create a dictionary for the current country in the vectorizer_dict
    vectorizer_dict[f"{country}"] = {}
    
    # Add the TfidfVectorizer object, preprocessor function, tokenizer function, and n-gram dictionary to the country's dictionary
    vectorizer_dict[f"{country}"]['vectorizer'] = tfidf_vectorizer
    vectorizer_dict[f"{country}"]['vectorizerPreprocessor'] = tfidf_vectorizer.build_preprocessor()
    vectorizer_dict[f"{country}"]['vectorizerTokenizer'] = tfidf_vectorizer.build_tokenizer()
    vectorizer_dict[f"{country}"]['terms_dict']  = buildNgramDictionary(tfidf_vectorizer, country)
    
##TIME : 7 mins

  0%|          | 0/28 [00:00<?, ?it/s]

## Apply SentenceRemover on entire DF to create short(quality) version of description 'short_desc'

In [9]:
short_desc_cache={} #dictionary to store 'description':'short_desc' for faster computation of short descriptions

In [10]:
tqdm.pandas()
df['short_desc'] = df.progress_apply(lambda x: removeTrashSentencesIDFv2(x['description'],vectorizer_dict[x['country_id']]), axis=1 )

  0%|          | 0/112056 [00:00<?, ?it/s]

In [ ]:
##TIME 5:51H

In [10]:
short_desc_cache={}#free up memory

## SentenceTransformer - Generating description embeddings for entire DF
- The pretrained model is https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2 with maximum number of input tokens (512)

In [10]:
#Instatiance SentenceTransformer
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
model.max_seq_length = 512 
print(f"Max seq length: {model.get_max_seq_length}")

Max seq length: <bound method SentenceTransformer.get_max_seq_length of SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)>


In [10]:
short_desc_embedding_cache={} #dictionary to store 'short_desc':'embedding' for faster computation of embeddings

In [15]:
def generate_emb(text):
    if text in short_desc_embedding_cache:#check if the embedding for short_desc has previously created 
            return short_desc_embedding_cache[text]
    else: 
        # Handling missing values
        if (text is None) or (text == "") or (pd.isna(text)):
            short_desc_embedding_cache[text] = np.zeros(512)#save copy to cache
            return np.zeros(512)
        else:
            embedding = model.encode(text)
            short_desc_embedding_cache[text] = embedding#save copy to cache
            return embedding

In [16]:
tqdm.pandas()
df['embedding'] = df.progress_apply(lambda x: generate_emb(x['short_desc']), axis=1 )

  0%|          | 0/112056 [00:00<?, ?it/s]

In [ ]:
##TIME 3:40 h

In [ ]:
short_desc_embedding_cache={}#.clear()

# Query semantic nearest neighbour for each embedding to identify Semantic/Temporal duplicates 
- Based on semantic neareast-neighbours search we used cosine similarity to take the most similar subset 
- On the selected subset we used custom method to remove non-duplicates offers and to assign category to pair as either Semantic or Temporal

In [18]:
#fix indexing
df = df.sort_values(['id']).reset_index(drop=True)

## Initialize Brute force indexing
- Brute force version of searching nearest neighbours in high-dimensional space provided by https://github.com/nmslib/hnswlib

In [19]:
# Set the dimensionality of the embeddings to 512 and set the random seed to 42
dim = 512
np.random.seed(42)

# Extract the embeddings from the "embedding" column of the DataFrame as a list
embeddings = df["embedding"].tolist()

# Get the number of elements in the dataset
num_elements = len(embeddings)

# Convert the embeddings to a numpy array
data = np.array(embeddings)

# Initialize a brute-force index with the cosine similarity metric and the specified dimensionality
bf_index = hnswlib.BFIndex(space='cosine', dim=dim)

# Initialize the index with the maximum number of elements
bf_index.init_index(max_elements=num_elements)

# Add the embeddings to the index
print("Adding batch of %d elements" % (len(data)))
bf_index.add_items(data)

# Print a message indicating that the index has been built
print("Indices built")

Adding batch of 112056 elements
Indices built


## Query NN for each embedding

In [23]:
def compute_jaccard(set1, set2):
    """Function to compute jaccard similarity based on words splitted by space"""
    if(set1 is None) or (set1 is None):
        return 0.0
    try:
        set1 = set(set1.split())
        set2 = set(set2.split())
        return len(set1.intersection(set2)) / len(set1.union(set2))
    except ZeroDivisionError:
        return 0.0
    

queryResult={
    'Left':[],#offer ID (smaller id of two)
    'Right' :[],#offer ID (bigger id of two)
    'cosSim': [] # List to store cosine similarity scores between offer embedding based on 'short_desc'
    
}

k = 43 # Number of nearest neighbors to query
total = len(df) # Total number of offers in the dataset

# Loop over each offer in the dataset
for index, row in tqdm(df.iterrows(), total=total):
    # Find k nearest neighbors using brute-force search (k-NN)
    labels_bf, distances_bf = bf_index.knn_query(row['embedding'], k)
    
    # Get embedding and 'description' of current offer
    embedding1 = df.iloc[index, -1]
    embedding1_desc = df.iloc[index, 2]
    
    # Loop over each of the k nearest neighbors
    for i in range(len(labels_bf[0])):
        # Get IDs of current offer and its neighbor
        offerID = index + 1
        neighbourOfferID = int(labels_bf[0][i] + 1)
        
        # Store IDs of offers in queryResult in a way that Left < Right
        if offerID > neighbourOfferID:
            queryResult['Left'].append(neighbourOfferID)
            queryResult['Right'].append(offerID)
        elif offerID < neighbourOfferID:
            queryResult['Left'].append(offerID)
            queryResult['Right'].append(neighbourOfferID)
        else:
            #skip if two IDs are the same
            continue
        
        # Get embedding and description of current neighbour
        embedding2 = df.iloc[labels_bf[0][i], -1]
        embedding2_desc = df.iloc[labels_bf[0][i], 2]
        
        
        # Compute cosine similarity score between offer embedding based on 'short_desc'
        cos_sim = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
        queryResult['cosSim'].append(cos_sim)
#TIME 40 mins     

## create df with nearest-neighbours pairs

In [21]:
df_nn = pd.DataFrame(queryResult)
df_nn=df_nn.drop_duplicates(subset=['Left','Right'])


## Assign category to pairs from semantic similarity search
- To ensure title semantic similarity we used the same model as previously https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2 with maximum number of input tokens (50)

### Partial duplicate identification
- We use LSH to identify parent offer (if exists) that contains all details (is extension of child)
- Then the child offer is a offer that is missing some information but does not have additional characteristics
- Every child offer is potential partial duplicate localized in subset of semantic duplicates

In [76]:
def compute_jaccard(set1, set2):
    """Function to compute jaccard similarity based on words splitted by space"""
    if(set1 is None) or (set1 is None):
        return 0.0
    try:
        set1 = set(set1.split())
        set2 = set(set2.split())
        return len(set1.intersection(set2)) / len(set1.union(set2))
    except ZeroDivisionError:
        return 0.0
    
    
def nMissing(text1,text2):
    """Method to measure how many more words are in text2"""
    text1=set(text1.split())
    text2=set(text2.split())
    isntext1nottext2 = text1.difference(text2)
    isntext2nottext1 = text2.difference(text1)
    if((len(isntext1nottext2) == 0) and (len(isntext2nottext1)>0)):
        #only interesting case when the text1(child offer) contains no more information then the parent have 
        return len(isntext2nottext1)
        
    else:
        return 0
    

# Define number of permutations and size of n-grams
permutations = 128
n_gram_size = 1

def get_forest(data, perms):
    minhash = [] # List to store MinHash objects for each offer's short description
    
    # Loop over each offer's short description in the dataset
    for text in data['description']:
        tokens = text.split(" ") # Split text into tokens (words)
        ngrams = [' '.join(tokens[i:i+n_gram_size]) for i in range(len(tokens)-n_gram_size-1)] # Generate n-grams of specified size
        m = MinHash(num_perm=perms) # Create MinHash object with specified number of permutations
        for s in ngrams:
            m.update(s.encode('utf8')) # Update MinHash object with each n-gram
        minhash.append(m) # Add MinHash object to list
        
    forest = MinHashLSHForest(num_perm=perms) # Create MinHash LSH Forest object with specified number of permutations
    
    # Add each offer's MinHash object to the LSH Forest
    for i, m in enumerate(minhash):
        forest.add(i, m)
        
    forest.index() # Index the MinHash LSH Forest
    
    return forest # Return the MinHash LSH Forest object


def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens =text.split(" ") # Split text into tokens (words)
    ngrams = [' '.join(tokens[i:i+n_gram_size]) for i in range(len(tokens)-n_gram_size-1)] # Generate n-grams of specified size from tokens
    
    m = MinHash(num_perm=perms) # Create MinHash object with specified number of permutations
    for s in ngrams:
        m.update(s.encode('utf8')) # Update MinHash object with each n-gram
    
    idx_array = np.array(forest.query(m, num_results)) # Query the MinHash LSH Forest for similar offers and return an array of their indices in the database
    
    if len(idx_array) == 0:
        return None # Return None if no similar offers were found
    
    # Compute Jaccard similarity between the query and each similar offer's 'short_desc_loc' and store in a list of tuples with offer IDs
    jaccard_results = []
    for idx in idx_array:
        jaccard_sim = compute_jaccard(database.iloc[idx]['description'], text) # Compute Jaccard similarity between the query and the similar offer's short description
        offer_id = database.iloc[idx]['id'] # Get the ID of the similar offer
        jaccard_results.append((offer_id, jaccard_sim)) # Append a tuple of the offer ID and its Jaccard similarity to the list
        
    jaccard_results = sorted(jaccard_results, key=lambda x: x[1], reverse=True) # Sort the list of similar offers by decreasing Jaccard similarity
    
    return jaccard_results # Return the list of similar offers with their IDs and Jaccard similarities


In [75]:
df_desc=df.dropna(subset=['description'])
df_desc=df_desc.reset_index(drop=True)
forest = get_forest(df_desc, permutations)

In [ ]:
# Initialize an empty list to store results
possiblePartial=[]
# Define the number of recommended(closest offer to query) offers
num_recommendations = 50

# Loop through each row of the dataframe
#check if offer have parent description
for index, row in tqdm(df_desc.iterrows(),total=df_desc.shape[0]):
    
    
    # Extract the query text and generate a set of predicted offers based on the LSH forest
    query = row['description']
    result = predict(query, df_desc, permutations, num_recommendations, forest)
    
    # Extract the ID and retrieval date of the current offer
    offerIDLeft = row['id']


    # Loop through the predicted offers and compare them to the current offer
    for res in result:
        
        # Extract the Jaccard similarity and ID of the predicted offer
        jaccard_sim = res[1]
        offerIDRight = res[0]
        
        rightDesc = df_desc[df_desc['id']==offerIDRight].reset_index(drop=True)['description'][0]
        if jaccard_sim<0.2:#pairs with lower jaccard will have to much differences
            continue
        else:
            nMiss = nMissing(query,rightDesc)#number of missing words in query
            if (nMiss<20) and (nMiss>0):#if its about the lenght of common job characteristics then assign offer id partial class
                possiblePartial.append(
                    {'offerIDPartial':offerIDLeft,
                        'offerIDExtension' :offerIDRight,
                        'type' :"PARTIAL",
                         'nMissing':nMiss
                        })
                

#TIME 1:40 h

In [ ]:
df_partial = pd.DataFrame(possiblePartial)

In [101]:
#save identified partial offer's id to list where there is at least two words missing
idPartial = df_partial[df_partial['nMissing']>=2]['offerIDPartial'].unique()

#### Initialize methods to assign category

In [78]:
from sentence_transformers import SentenceTransformer
model_title = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
model_title.max_seq_length = 50
emb_cache = {}#dictionary to store 'title':'title_embedding'


def get_class_cos_1(row, cos_title_threshold_temp=0.65, cos_title_threshold_sem=0.58): 
    offer1 = df[df['id']==row['Left']].reset_index(drop=True)
    offer2 = df[df['id']==row['Right']].reset_index(drop=True)
    title1 = offer1['title'][0]
    title2 = offer2['title'][0]
    
    same_or_nan_num = 0
    for feature in ['location', 'country_id', 'company_name']:
        if ((offer1[feature][0] == offer2[feature][0]) or (offer1[feature][0] is np.nan and offer2[feature][0] is np.nan)):
            same_or_nan_num+=1
        
    if same_or_nan_num==3:
        if title1 in emb_cache:
            emb1 = emb_cache.get(title1)
        else:
            emb1 = model_title.encode(title1)
            emb_cache[title1] = emb1
        if title2 in emb_cache:
            emb2 = emb_cache.get(title2)
        else:
            emb2 = model_title.encode(title2)
            emb_cache[title2] = emb2
            
        if offer1['retrieval_date'][0] == offer2['retrieval_date'][0]:
            if ((offer1['description'][0]==offer2['description'][0]) and (title1==title2)):
                return 'FULL'
            else:
                if np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2)) >= cos_title_threshold_sem:
                    return 'SEMANTIC'
                else:
                    return 'NON-DUPLICATE'
        else:
            if np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2)) >= cos_title_threshold_temp:
                return 'TEMPORAL'
            else:
                return 'NON-DUPLICATE'
    else:
        return 'NON-DUPLICATE'


def get_class(row, cos_title_threshold_temp=0.65, cos_title_threshold_sem=0.56): 
    """Filter method prepared based on many analysis how to best assign duplicate category"""
    # Get the left and right offers from the DataFrame
    offer1 = df[df['id']==row['Left']].reset_index(drop=True)
    offer2 = df[df['id']==row['Right']].reset_index(drop=True)
    
    # Get the titles of the left and right offers
    title1 = offer1['title'][0]
    title2 = offer2['title'][0]
    
    # Count the number of features that are the same or both are NaN
    same_or_nan_num = 0
    for feature in ['location', 'country_id', 'company_name']:
        if ((offer1[feature][0] == offer2[feature][0]) or (offer1[feature][0] is np.nan and offer2[feature][0] is np.nan)):
            same_or_nan_num += 1
        
    # If all three features are the same or both are NaN
    if same_or_nan_num == 3:
        # Compute the embeddings for the left and right offer titles using the model_title
        if title1 in emb_cache:
            emb1 = emb_cache.get(title1)
        else:
            emb1 = model_title.encode(title1)
            emb_cache[title1] = emb1
        if title2 in emb_cache:
            emb2 = emb_cache.get(title2)
        else:
            emb2 = model_title.encode(title2)
            emb_cache[title2] = emb2
            
        # If the retrieval dates are the same
        if offer1['retrieval_date'][0] == offer2['retrieval_date'][0]:
            # If the descriptions and titles are the same
            if ((offer1['description'][0]==offer2['description'][0]) and (title1==title2)):
                return 'FULL'
            # If the cosine similarity between the embeddings of the titles is above the semantic threshold
            else:
                if np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2)) >= cos_title_threshold_sem:
                    if(row['Left'] in idPartial) or(row['Right'] in idPartial):
                        #if pair contains id that were identified as potential partial duplicate assign appropriate category
                        return "PARTIAL"
                    else:
                        return 'SEMANTIC'
                else:
                    return 'NON-DUPLICATE'
        # If the retrieval dates are different
        else:
            # If the cosine similarity between the embeddings of the titles is above the temporal threshold
            if np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2)) >= cos_title_threshold_temp:
                return 'TEMPORAL'
            else:
                return 'NON-DUPLICATE'
    # If any of the features are different
    else:
        return 'NON-DUPLICATE'

In [51]:
#separate df for offers with cos similarity ==1
#take only semantic duplicate from cosine==1 as by using short_desc temporal duplicate contains many false positives
df_nn_cos_1 = df_nn[df_nn['cosSim']==1]
#filter only semantic/temporal pairs
tqdm.pandas()
df_nn_cos_1['Class'] = df_nn_cos_1.progress_apply( lambda x: get_class_cos_1(x),axis=1)
df_nn_cos_1 = df_nn_cos_1[df_nn_cos_1['Class'].isin(['SEMANTIC'])]
df_nn_cos_1.drop('cosSim',axis=1,inplace=True)
#TIME 8 mins 

  0%|          | 0/349503 [00:00<?, ?it/s]

In [80]:
#cosine similarity threshold for temporal/semantic pair
min_similarity_threshold_semantic = 0.35
min_similarity_threshold_temporal = 0.48

#reduce pairs of nearest neighbours search 
df_nn=df_nn[(df_nn['cosSim']>=min_similarity_threshold_semantic)&(df_nn['cosSim']<1)]

total_pairs = df_nn.shape[0]

Class = [] #list to store assigned category for entire semantic neareast-neighbours dataframe
# Loop over each pair in the dataset
for index, row in tqdm(df_nn.iterrows(), total=total_pairs):
    offer1ID = row['Left']
    offer2ID = row['Right']
    
    offer1Row = df[df['id']==offer1ID].reset_index(drop=True)
    offer2Row = df[df['id']==offer2ID].reset_index(drop=True)
    
    
    if offer1Row.iat[0,6]==offer2Row.iat[0, 6]:#compare offer's retrieval dates
        #semantic threshold above include all pairs 
        Class.append(get_class(row))
    else:
        #for temporal pairs threshold needs to be check as there are pairs with cosine belowe 0.5
        if (row['cosSim']>=min_similarity_threshold_temporal):
            Class.append(get_class(row))
        else:
            Class.append("NON-DUPLICATE")
            continue
    #TIME 4h

  0%|          | 0/3003241 [00:00<?, ?it/s]

In [81]:
df_nn['Class']=Class

In [83]:
df_nn=df_nn[df_nn['Class'].isin(['TEMPORAL','SEMANTIC',"PARTIAL"])]

In [85]:
df_nn=df_nn.drop('cosSim',axis=1)

## LSH (Locality Sensitive Hashing) to identify partial duplicates
- During early submissions we used LSH to identify full duplicates, what we found out was that this approach gave as a chance to discover partial/semantic duplicates with high precision
- The approach uses MinHashLSHForest from datasketch library

In [18]:
#Drop NaN short_desc rows as they are not supported in methods below
df_short= df.dropna(subset=['short_desc'])

In [19]:
#To ensure higher probability of match the content from 'location' column was added to 'short_desc' -> so that 'short_desc_loc' column was made
df_short['short_desc_loc'] = df_short.apply(lambda x : x['location']+" "+x['short_desc'] if isinstance(x['location'],str)  else x['short_desc'],axis=1)

In [20]:
# Define number of permutations and size of n-grams
permutations = 128
n_gram_size = 5

def get_forest(data, perms):
    minhash = [] # List to store MinHash objects for each offer's short description
    
    # Loop over each offer's short description in the dataset
    for text in data['short_desc_loc']:
        tokens = text.split(" ") # Split text into tokens (words)
        ngrams = [' '.join(tokens[i:i+n_gram_size]) for i in range(len(tokens)-n_gram_size-1)] # Generate n-grams of specified size
        m = MinHash(num_perm=perms) # Create MinHash object with specified number of permutations
        for s in ngrams:
            m.update(s.encode('utf8')) # Update MinHash object with each n-gram
        minhash.append(m) # Add MinHash object to list
        
    forest = MinHashLSHForest(num_perm=perms) # Create MinHash LSH Forest object with specified number of permutations
    
    # Add each offer's MinHash object to the LSH Forest
    for i, m in enumerate(minhash):
        forest.add(i, m)
        
    forest.index() # Index the MinHash LSH Forest
    
    return forest # Return the MinHash LSH Forest object


def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens =text.split(" ") # Split text into tokens (words)
    ngrams = [' '.join(tokens[i:i+n_gram_size]) for i in range(len(tokens)-n_gram_size-1)] # Generate n-grams of specified size from tokens
    
    m = MinHash(num_perm=perms) # Create MinHash object with specified number of permutations
    for s in ngrams:
        m.update(s.encode('utf8')) # Update MinHash object with each n-gram
    
    idx_array = np.array(forest.query(m, num_results)) # Query the MinHash LSH Forest for similar offers and return an array of their indices in the database
    
    if len(idx_array) == 0:
        return None # Return None if no similar offers were found
    
    # Compute Jaccard similarity between the query and each similar offer's 'short_desc_loc' and store in a list of tuples with offer IDs
    jaccard_results = []
    for idx in idx_array:
        jaccard_sim = compute_jaccard(database.iloc[idx]['short_desc_loc'], text) # Compute Jaccard similarity between the query and the similar offer's short description
        offer_id = database.iloc[idx]['id'] # Get the ID of the similar offer
        jaccard_results.append((offer_id, jaccard_sim)) # Append a tuple of the offer ID and its Jaccard similarity to the list
        
    jaccard_results = sorted(jaccard_results, key=lambda x: x[1], reverse=True) # Sort the list of similar offers by decreasing Jaccard similarity
    
    return jaccard_results # Return the list of similar offers with their IDs and Jaccard similarities

def get_class_lsh(IDLeft,IDRight, cos_title_threshold_temp=0.65, cos_title_threshold_sem=0.58): 
    """The same Filter method as above prepared based on many analysis how to best assign duplicate category
    It takes different parameters and is used to only check the df with partial/semantic duplicates.
    The non-duplicate offers can be filter out if the result is different that 'SEMANTIC' """
    offer1 = df[df['id']==IDLeft].reset_index(drop=True)
    offer2 = df[df['id']==IDRight].reset_index(drop=True)
    title1 = offer1['title'][0]
    title2 = offer2['title'][0]
    
    same_or_nan_num = 0
    for feature in ['location', 'country_id', 'company_name']:
        if ((offer1[feature][0] == offer2[feature][0]) or (offer1[feature][0] is np.nan and offer2[feature][0] is np.nan)):
            same_or_nan_num+=1
        
    if same_or_nan_num==3:
        if title1 in emb_cache:
            emb1 = emb_cache.get(title1)
        else:
            emb1 = model_title.encode(title1)
            emb_cache[title1] = emb1
        if title2 in emb_cache:
            emb2 = emb_cache.get(title2)
        else:
            emb2 = model_title.encode(title2)
            emb_cache[title2] = emb2
            
        if offer1['retrieval_date'][0] == offer2['retrieval_date'][0]:
            if ((offer1['description'][0]==offer2['description'][0]) and (title1==title2)):
                return 'FULL'
            else:
                if np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2)) >= cos_title_threshold_sem:
                    return 'SEMANTIC'
                else:
                    return 'NON-DUPLICATE'
        else:
            if np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2)) >= cos_title_threshold_temp:
                return 'TEMPORAL'
            else:
                return 'NON-DUPLICATE'
    else:
        return 'NON-DUPLICATE'

### Creation of LSHForest for each language

In [21]:
# Create a dictionary to hold the forests for each country
forest_dict={}

# Loop through all unique country IDs in the dataset
for country in tqdm(df_short.country_id.unique()):
    
    # Filter the dataframe to only include offers from the current country
    df_country = df_short.query(f"country_id == '{country}'").reset_index(drop=True) 
    
    # Generate the forest for the current country's offers
    forest_country = get_forest(df_country, permutations)
    
    # Add the current country's dataframe and forest to the dictionary
    forest_dict[f"{country}"]={}
    forest_dict[f"{country}"]['df_country'] = df_country
    forest_dict[f"{country}"]['forest'] = forest_country
    
#TIME 4 min

  0%|          | 0/31 [00:00<?, ?it/s]

## Predict partial/semantic duplicates based on query result

In [30]:
# Initialize an empty list to store results
LSHResult=[]
# Define the number of recommended(closest offer to query) offers
num_recommendations = 100

# Loop through each row of the dataframe
for index, row in tqdm(df_short.iterrows(),total=df_short.shape[0]):
    # Retrieve the LSH forest and country-specific dataframe for the current row
    countryForest = forest_dict[row['country_id']]['forest']
    df_country = forest_dict[row['country_id']]['df_country']
    
    # Extract the query text and generate a set of predicted offers based on the LSH forest
    query = row['short_desc_loc']
    result = predict(query, df_country, permutations, num_recommendations, countryForest)
    
    # Extract the ID and retrieval date of the current offer
    offerIDLeft = row['id']
    ret_date_offerLeft= row['retrieval_date']

    # Loop through the predicted offers and compare them to the current offer
    for res in result:
        
        # Extract the Jaccard similarity and ID of the predicted offer
        jaccard_sim = res[1]
        offerIDRight = res[0]
        
        # Extract the row of the predicted offer from the country-specific dataframe
        row_to_check = df_country.query(f"id == {offerIDRight}")
        
        # If the predicted offer is the same as the current offer, skip it
        if (offerIDLeft==offerIDRight):
            continue
        
        # If the predicted offer has a Jaccard similarity between 0.6 and 0.8, and the same retrieval date as the current offer, add it to the LSHResult list
        elif((jaccard_sim <0.8) and(jaccard_sim >=0.6)):
            ret_date_offerRight = row_to_check['retrieval_date'].iloc[0]
            
            if(ret_date_offerLeft==ret_date_offerRight):#THE SAME DATE
                #set valid index
                if (offerIDLeft<offerIDRight):
                    LSHResult.append(
                    {'offerIDLeft':offerIDLeft,
                        'offerIDRight' :offerIDRight,
                        'type' :"SEMANTIC",
                         'Class':get_class_lsh(offerIDLeft,offerIDRight)
                        })
                else:
                    LSHResult.append(
                    {'offerIDLeft':offerIDRight,
                        'offerIDRight' :offerIDLeft,
                        'type' :"SEMANTIC",
                         'Class':get_class_lsh(offerIDLeft,offerIDRight)
                        })
        
        # If the predicted offer has a Jaccard similarity between 0.8 and 1, and the same retrieval date as the current offer, add it to the LSHResult list
        elif((jaccard_sim >0.8) and(jaccard_sim <1)):
            ret_date_offerRight = row_to_check['retrieval_date'].iloc[0]
            
            if(ret_date_offerLeft==ret_date_offerRight):#THE SAME DATE
                #set valid index
                if (offerIDLeft<offerIDRight):
                    LSHResult.append(
                    {'offerIDLeft':offerIDLeft,
                        'offerIDRight' :offerIDRight,
                        'type' :"PARTIAL",
                         'Class':get_class_lsh(offerIDLeft,offerIDRight)
                        })
                else:
                    LSHResult.append(
                    {'offerIDLeft':offerIDRight,
                        'offerIDRight' :offerIDLeft,
                        'type' :"PARTIAL",
                         'Class':get_class_lsh(offerIDLeft,offerIDRight)
                        })
        
        # If the predicted offer has a Jaccard similarity less than 0.6 or retrieval date different from current offer, skip it
        else:
            pass
#TIME 40 mins

  0%|          | 0/112035 [00:00<?, ?it/s]

In [31]:
#Store result inside separate df
df_lsh_partial_semantic = pd.DataFrame(LSHResult)
#assign class based on previous type and Class from get_class method; duplicates with assign SEMANTIC are the ones to save
df_lsh_partial_semantic['Class']=df_lsh_partial_semantic.apply(lambda x: x['type'] if x['Class']=='SEMANTIC' else 'NON-DUPLICATE',axis=1)
#filter only duplicate pairs
df_lsh_partial_semantic=df_lsh_partial_semantic[df_lsh_partial_semantic['Class'].isin(['SEMANTIC','PARTIAL'])]
#change columns names to match other dfs
df_lsh_partial_semantic=df_lsh_partial_semantic.rename(columns={'offerIDLeft':'Left','offerIDRight':'Right'})
#remove redudant column
df_lsh_partial_semantic.drop(['type'],inplace=True,axis=1)

# Identification of full/temporal duplicates

In [89]:
df['description'] = df['description'].str.lower()
df['title'] = df['title'].str.lower()
df['company_name'] = df['company_name'].str.lower()
df['location'] = df['location'].str.lower()
# Count the number of occurrences of each unique 'title' value in the DataFrame 'df'
title_count = df['title'].value_counts()
# Keep only the values in 'title_count' that appear more than once
title_count = title_count[title_count>1]

# Count the number of occurrences of each unique 'description' value in the DataFrame 'df'
desc_count = df['description'].value_counts()
# Keep only the values in 'desc_count' that appear more than once
desc_count = desc_count[desc_count>1]

In [90]:
# Select all rows from df that have a 'title' value in title_count and a 'description' value in desc_count
df_dup = df[(df['title'].isin(title_count.index)) & (df['description'].isin(desc_count.index))]
# Reset the index of df_dup to start from 0 and drop the old index
df_dup = df_dup.reset_index(drop=True)

In [91]:
# Initialize empty lists to store information about duplicates
Left = []
Right = []
Class = []

# Use tqdm to display a progress bar during the loop
with tqdm(total=len(df_dup)) as pbar:
    # Iterate over the rows of df_dup
    for row in df_dup.iterrows():
        # Update the progress bar
        pbar.update(1)
        # Select the offer from the current row
        offer = row[1]
        # Find all offers in df with the same title as the current offer
        to_check = df[df['title']==offer['title']]
        # Iterate over these offers to find duplicates
        for row in to_check.iterrows():
            # Select the offer to check for duplication
            offer_check = row[1]
            # Check if the offer and offer_check have identical title, description, and different id
            if offer['description']==offer_check['description']\
            and offer['title']==offer_check['title']\
            and offer['id']!=offer_check['id']:
                # If duplicates are found, store their ids in Left and Right lists
                Left.append(offer['id'])
                Right.append(offer_check['id'])
                # Classify the type of duplicate based on whether they have the same retrieval date or not
                if offer['retrieval_date']==offer_check['retrieval_date']:
                    Class.append("FULL")
                else:
                    Class.append("TEMPORAL")
#TIME 6 mins

  0%|          | 0/54081 [00:00<?, ?it/s]

In [21]:
def set_higher_on_the_right(df_uncharted):
    """Method to change the index inside df; id1<id2"""
    df = df_uncharted.copy(deep=True)
    i = 0
    with tqdm(total=len(df)) as pbar:
        for row in df.iterrows():
            pbar.update(1)
            if row[1]['Right'] < row[1]['Left']:
                left = row[1]['Right']
                right = row[1]['Left']
                df.iat[i, 0] = left
                df.iat[i, 1] = right
            i+=1
    return df

def is_right(df):
    '''Test method to check duplicates.csv assumptions'''
    left = df['Left'].to_list()
    right = df['Right'].to_list()
    for i in range(len(left)):
        if left[i] >= right[i]:
            raise Exception(f"On index {i} left value is greater or equal than right.")
            
    if df[['Left', 'Right']].duplicated().sum()>0:
        raise Exception(f"There is duplicated rows.")
    print("Everything ok.")

In [100]:
df_full_temporal = pd.DataFrame({"Left": Left, "Right": Right, "Class": Class})
df_full_temporal = set_higher_on_the_right(df_full_temporal)
#TIME 6 min

  0%|          | 0/714936 [00:00<?, ?it/s]

In [101]:
df_full_temporal.drop_duplicates(subset=['Left', 'Right'], inplace=True)

In [142]:
#assign 'PARTIAL' category for only 'FULL' pairs if any index from pair is possiblePartial (idPartial)
df_full_temporal['Class'] = df_full_temporal.apply(lambda x : 'PARTIAL' if (((x['Left'] in idPartial) or (x['Right'] in idPartial)) and (x['Class'] in ['FULL'])) else x['Class'],axis=1)


## Create duplicates.csv from many df 

In [147]:
#the order of the dfs is important as it mirrors the confidence of indetified pairs
df_result = pd.concat([df_full_temporal ,df_lsh_partial_semantic,df_nn,df_nn_cos_1], axis=0)
df_result.drop_duplicates(subset=['Left', 'Right'], keep='first', inplace=True)
df_result.reset_index(drop=True, inplace=True)

In [153]:
df_result.to_csv('./duplicates.csv',index=False,header=False)